In [ ]:
# The following analysis was proposed in discussions with Khurana group: 
# https://mail.google.com/mail/u/0/#inbox/QgrcJHrnvrtLdKxwGgQthdBxLwhhXMNvctg

In [ ]:
# TODO 
# 1. for the 500+ homozygous deleted enhancers in 1000G (only a handful TOPMED events are not included), obtain the khurana scores by merging with ENHANCERS
# 2. concatenate disease enhancers, and training positives 
# 3. train-test split 
# 4. train, and compute PR curve on test set ( experiments/germline-model/chen-et-al-2022/predict-enhancer-constraint-using-chen-and-deletions.4.ipynb )
# 5. ask whether scores from new models, which are class probabilities, are better calibrated than Khurana’s probabilities ( https://www.nature.com/articles/s41588-023-01373-3/figures/13 )

## Get a set of enhancers known to be constrained or not, together with their Chen scores and Khurana scores

In [1]:
CONSTRAINT_TOOLS = '/scratch/ucgd/lustre-work/quinlan/u6018199/constraint-tools'
CONSTRAINT_TOOLS_DATA = '/scratch/ucgd/lustre-work/quinlan/data-shared/constraint-tools'

In [2]:
import pandas as pd

pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 10)

def aggregate_over_windows(df): 
  group_columns = set(df.columns) - set(['negative_new_chen_score_window']) 
  group_columns = list(group_columns)
  groups = df.groupby(group_columns)
  aggregation_functions = {'negative_new_chen_score_window': ['min', 'count']}
  aggregated = groups.agg(aggregation_functions)
  
  df = aggregated.reset_index()
  df.columns = [' '.join(col[::-1]).strip() for col in df.columns.values]
  return df

def read_disease_enhancers_intersect_chen_windows():
  # {CONSTRAINT_TOOLS}/download-process-data/khurana/README.md
  df = pd.read_csv(
    "{}/khurana/disease-enhancers-intersect-chen-windows.bed".format(CONSTRAINT_TOOLS_DATA), 
    sep = '\t',
    names = [
        'chrom_enhancer', 'start_enhancer', 'end_enhancer', 
        'disease', 'enhancer_predicted_LoF_tolerance_prob', 'enhancer_predicted_LoF_tolerance_status', 
        'chrom_window', 'start_window', 'end_window', 
        'new_chen_score_window'
    ]
  )
  df = df.drop_duplicates() 
  df['negative_new_chen_score_window'] = -df['new_chen_score_window']
  df = df.rename(columns={
    'chrom_enhancer': 'chrom', 
    'start_enhancer': 'start', 
    'end_enhancer': 'end'
  })
  df['enhancer_predicted_constraint_prob_khurana'] = 1.0 - df['enhancer_predicted_LoF_tolerance_prob']
  df = df.drop(columns=[
    'new_chen_score_window',
    'chrom_window', 'start_window', 'end_window',
    'disease', 'enhancer_predicted_LoF_tolerance_prob', 'enhancer_predicted_LoF_tolerance_status'
  ])
  df['truly constrained'] = True
  df['tag'] = 'disease_enhancers'
  return df 

aggregate_over_windows(read_disease_enhancers_intersect_chen_windows())

,end,tag,start,truly constrained,enhancer_predicted_constraint_prob_khurana,chrom,min negative_new_chen_score_window,count negative_new_chen_score_window
0,2160634,disease_enhancers,2159834,True,0.336234,chr12,-3.402956,2
1,5228370,disease_enhancers,5225970,True,0.506432,chr11,-0.612337,4
2,5236370,disease_enhancers,5236170,True,0.157999,chr11,-0.608309,1
3,6366400,disease_enhancers,6364600,True,0.300163,chr9,-0.233282,3
4,6472369,disease_enhancers,6471569,True,0.543985,chr7,-0.805666,1
...,...,...,...,...,...,...,...,...
68,170834490,disease_enhancers,170832090,True,0.678984,chr2,-2.054150,1
69,173915462,disease_enhancers,173915262,True,0.074476,chr1,-1.790974,1
70,177993199,disease_enhancers,177992999,True,0.165411,chr5,-3.607152,1
71,199920277,disease_enhancers,199919277,True,0.405388,chr2,-1.605622,1


In [8]:
def read_pacbio_deleted_enhancers_intersect_chen_windows():
  # download-process-data/khurana/README.md
  df = pd.read_csv(
      "{}/khurana/pacbio-deleted-enhancers-intersect-chen-windows.bed".format(CONSTRAINT_TOOLS_DATA), 
      sep = '\t',
      names = [
          'chrom_enhancer', 'start_enhancer', 'end_enhancer', 
          'family', 'enhancer_predicted_LoF_tolerance_prob', 'enhancer_predicted_LoF_tolerance_status', 
          'chrom_window', 'start_window', 'end_window', 
          'new_chen_score_window'
      ]
  )
  df = df.drop_duplicates() 
  df['negative_new_chen_score_window'] = -df['new_chen_score_window']
  df['enhancer_predicted_constraint_prob_khurana'] = 1.0 - df['enhancer_predicted_LoF_tolerance_prob']
  df = df.drop(columns=[
    'new_chen_score_window',
    'chrom_window', 'start_window', 'end_window',
    'family', 'enhancer_predicted_LoF_tolerance_prob', 'enhancer_predicted_LoF_tolerance_status'
  ])
  df = df.rename(columns={
    'chrom_enhancer': 'chrom', 
    'start_enhancer': 'start', 
    'end_enhancer': 'end'
  })
  df['truly constrained'] = False
  df['tag'] = 'pacbio_deleted'
  return df 

aggregate_over_windows(read_pacbio_deleted_enhancers_intersect_chen_windows())

,end,tag,start,truly constrained,enhancer_predicted_constraint_prob_khurana,chrom,min negative_new_chen_score_window,count negative_new_chen_score_window
0,5785668,pacbio_deleted,5785468,False,0.045060,chr2,2.210384,1
1,6970081,pacbio_deleted,6969081,False,0.323341,chr17,-2.492414,1
2,69153682,pacbio_deleted,69152682,False,0.304378,chr4,1.650749,2
3,121543953,pacbio_deleted,121543753,False,0.289212,chr3,-1.011635,1


In [10]:
# get enhancers that are homozygously deleted in 1000 Genomes Project
def read_lof_tolerant_enhancers_intersect_chen_windows():
  # download-process-data/khurana/README.md
  df = pd.read_csv(
      "{}/khurana/lof-tolerant-enhancers-intersect-chen-windows.bed".format(CONSTRAINT_TOOLS_DATA), 
      sep = '\t',
      names = [
          'chrom_enhancer_hg38', 'start_enhancer_hg38', 'end_enhancer_hg38', 
          'enhancer_hg19', 'unknown',
          'chrom_window', 'start_window', 'end_window', 
          'new_chen_score_window'
      ]
  )
  df = df.drop_duplicates() 
  df['negative_new_chen_score_window'] = -df['new_chen_score_window']
  df = df.drop(columns=[
    'new_chen_score_window',
    'chrom_window', 'start_window', 'end_window',
    'enhancer_hg19', 'unknown'
  ])
  df = df.rename(columns={
    'chrom_enhancer_hg38': 'chrom', 
    'start_enhancer_hg38': 'start', 
    'end_enhancer_hg38': 'end'
  })
  df['truly constrained'] = False
  df['tag'] = 'lof_tolerant'
  return df 

aggregate_over_windows(read_lof_tolerant_enhancers_intersect_chen_windows())

,chrom,start,truly constrained,end,tag,min negative_new_chen_score_window,count negative_new_chen_score_window
0,chr1,831220,False,832820,lof_tolerant,-1.248459,1
1,chr1,1508220,False,1509820,lof_tolerant,-1.731383,2
2,chr1,8124140,False,8124940,lof_tolerant,-0.841769,1
3,chr1,12684990,False,12685190,lof_tolerant,-1.403148,2
4,chr1,16104305,False,16105505,lof_tolerant,0.845290,1
...,...,...,...,...,...,...,...
505,chr9,126908121,False,126908321,lof_tolerant,-1.462233,1
506,chr9,129765721,False,129766721,lof_tolerant,-1.356569,1
507,chr9,129766921,False,129767321,lof_tolerant,-1.356569,1
508,chr9,132748213,False,132748813,lof_tolerant,-2.111835,1


In [ ]:
# TODO: 
# request scores for ALL homozygously deleted enhancers (or the network features for those examples)


In [11]:
# get low-lof-tolerance enhancers
def read_low_lof_tolerance_enhancers_intersect_chen_windows():
  # download-process-data/khurana/README.md
  df = pd.read_csv(
      "{}/khurana/low-lof-tolerance-enhancers-intersect-chen-windows.bed".format(CONSTRAINT_TOOLS_DATA), 
      sep = '\t',
      names = [
          'chrom_enhancer_hg38', 'start_enhancer_hg38', 'end_enhancer_hg38', 
          'enhancer_hg19', 'unknown',
          'chrom_window', 'start_window', 'end_window', 
          'new_chen_score_window'
      ]
  )
  df = df.drop_duplicates() 
  df['negative_new_chen_score_window'] = -df['new_chen_score_window']
  df = df.drop(columns=[
    'new_chen_score_window',
    'chrom_window', 'start_window', 'end_window',
    'enhancer_hg19', 'unknown'
  ])
  df = df.rename(columns={
    'chrom_enhancer_hg38': 'chrom', 
    'start_enhancer_hg38': 'start', 
    'end_enhancer_hg38': 'end'
  })
  df['truly constrained'] = True
  df['tag'] = 'low_lof_tolerance'
  return df 

aggregate_over_windows(read_low_lof_tolerance_enhancers_intersect_chen_windows())

,chrom,start,truly constrained,end,tag,min negative_new_chen_score_window,count negative_new_chen_score_window
0,chr1,61587728,True,61589928,low_lof_tolerance,-3.667483,3
1,chr1,87355917,True,87357117,low_lof_tolerance,-2.971624,3
2,chr1,169941659,True,169943659,low_lof_tolerance,-0.732262,2
3,chr10,100612643,True,100615043,low_lof_tolerance,-1.667674,4
4,chr10,100686843,True,100689243,low_lof_tolerance,-1.212765,4
...,...,...,...,...,...,...,...
40,chr7,31361986,True,31364186,low_lof_tolerance,-1.247859,4
41,chr7,54568507,True,54569507,low_lof_tolerance,-1.270480,2
42,chr7,97011888,True,97013488,low_lof_tolerance,-0.365397,2
43,chr8,98415972,True,98417172,low_lof_tolerance,0.620860,1


In [15]:
def create_labeled_enhancers(): 
  df = pd.concat(
    [
      aggregate_over_windows(read_disease_enhancers_intersect_chen_windows()),
      aggregate_over_windows(read_pacbio_deleted_enhancers_intersect_chen_windows()),
      aggregate_over_windows(read_lof_tolerant_enhancers_intersect_chen_windows()),
      aggregate_over_windows(read_low_lof_tolerance_enhancers_intersect_chen_windows())  
    ], 
    sort=True
  )
  return df 
  df = df.reset_index(drop=False)

  df = df.rename(columns={'index': 'enhancer_id'})
  new_order = df.columns[1:].tolist() + ['enhancer_id']
  df = df.reindex(columns=new_order)
  
  print('number of enhancers that are truly constrained (True) or not (False):') 
  print(df['truly constrained'].value_counts())

  return df 

create_labeled_enhancers()

,chrom,count negative_new_chen_score_window,end,enhancer_predicted_constraint_prob_khurana,min negative_new_chen_score_window,start,tag,truly constrained
0,chr12,2,2160634,0.336234,-3.402956,2159834,disease_enhancers,True
1,chr11,4,5228370,0.506432,-0.612337,5225970,disease_enhancers,True
2,chr11,1,5236370,0.157999,-0.608309,5236170,disease_enhancers,True
3,chr9,3,6366400,0.300163,-0.233282,6364600,disease_enhancers,True
4,chr7,1,6472369,0.543985,-0.805666,6471569,disease_enhancers,True
...,...,...,...,...,...,...,...,...
40,chr7,4,31364186,NaN,-1.247859,31361986,low_lof_tolerance,True
41,chr7,2,54569507,NaN,-1.270480,54568507,low_lof_tolerance,True
42,chr7,2,97013488,NaN,-0.365397,97011888,low_lof_tolerance,True
43,chr8,1,98417172,NaN,0.620860,98415972,low_lof_tolerance,True


## [TODO] Using Chen zscore to predict whether an enhancer is critical or not, using Khurana score, and using Chen-Khurana score

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

def plot_zscore_distributions(df):
  ax = sns.histplot(data=df, x='min negative_new_chen_score_window', kde=True, bins=25, hue='truly constrained')
  ax.set_xlabel('Chen z-score of enhancer')
  ax.set_ylabel('Number of enhancers')
  legend = ax.get_legend()
  legend.set_title('Enhancer truly constrained?')
  fig = plt.gcf()
  fig.set_size_inches(10, 5)

plot_zscore_distributions(ENHANCERS)

In [ ]:
def predict_constraint_without_deletions(df, zscore_threshold): 
  score = 'min negative_new_chen_score_window'
  df = df[[score, 'truly constrained']].copy()
  df['predicted to be constrained'] = df[score] < zscore_threshold
  return df

predict_constraint_without_deletions(ENHANCERS, zscore_threshold=-2)

In [ ]:
def predict_constraint_using_deletions(df, zscore_threshold): 
  score = 'min negative_new_chen_score_window'
  df = df[[
    score, 
    'truly constrained', 
    'enhancer is homozygously deleted in topmed'
  ]].copy()
  df['predicted to be constrained'] = (
    (df[score] < zscore_threshold) & 
    (df['enhancer is homozygously deleted in topmed'] == False)
  )
  return df

pd.set_option('display.max_rows', 10)

predict_constraint_using_deletions(ENHANCERS, zscore_threshold=-2)

In [ ]:
def compute_precision_recall(df, zscore_threshold, predict_constraint, log=False): 
  df = predict_constraint(df, zscore_threshold)
  
  contingency_table = pd.crosstab(
    df['predicted to be constrained'], 
    df['truly constrained']
  )

  if log:   
    print(zscore_threshold)
    print(contingency_table)

  tp = contingency_table.loc[True, True]
  fp = contingency_table.loc[True, False]
  fn = contingency_table.loc[False, True]
  precision = tp / (tp + fp)
  recall = tp / (tp + fn)

  return precision, recall
  
compute_precision_recall(ENHANCERS, zscore_threshold=-2, predict_constraint=predict_constraint_without_deletions, log=True)

In [ ]:
def plot_precision_recall(df, predict_constraint, label): 
  zscore_thresholds = np.arange(-4.5, 6.5, 0.1)
  precision_recall = [
    compute_precision_recall(df, zscore_threshold, predict_constraint)
    for zscore_threshold in zscore_thresholds
  ]
  precisions, recalls = zip(*precision_recall)
  plt.plot(recalls, precisions, label=label)

def plot_precision_recall_wrapper(df): 
  plot_precision_recall(df, predict_constraint_without_deletions, label='without using topmed deletions')
  plot_precision_recall(df, predict_constraint_using_deletions, label='using topmed deletions')
  plt.xlabel('Recall')
  plt.ylabel('Precision')
  plt.legend()
  fig = plt.gcf()
  fig.set_size_inches(10, 5)
  plt.title('Predicting enhancer constraint')

plot_precision_recall_wrapper(ENHANCERS)